## Workflow:
1. The bot greets the user and shows him menu(For now lets not consider bot showing menu. The bot greets the user and asks for order.)
2. User enters his query. The query can be anything:
    - The user might enter a food name only(Burger).
    - The user might query with full sentence for order like: I'd like to have a Pizza please.
    - The user might query to ask for the price.
    - The user might query to ask for recipe of a food item.

In [1]:
import os
from dotenv import load_dotenv
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnableMap, RunnablePassthrough
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import CSVLoader
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, AgentType
from langchain.tools.retriever import create_retriever_tool
from langchain_text_splitters import  RecursiveCharacterTextSplitter
from langchain.chains import LLMChain, create_tagging_chain_pydantic
from langchain_core.tools import Tool
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
load_dotenv()
llm = ChatOpenAI(temperature = 0, model_name = 'gpt-3.5-turbo')


In [2]:
loader = CSVLoader('./Datasets/menu.csv')
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000,chunk_overlap = 200)
splits = text_splitter.split_documents(docs)
# # embeddings = GoogleGenerativeAIEmbeddings(model = 'models/embedding-001')
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
from database import vector_store
# vector_store = Chroma.from_documents(documents=splits, embedding=embeddings,collection_name='menu_data' ,persist_directory = './Chroma/')

In [3]:
# from langchain.vectorstores.milvus import Milvus
# load_dotenv()
# host = os.getenv('HOST')
# port = os.getenv('PORT')
# # vector_store = Milvus(embeddings,connection_args={'host':host,'port':port},collection_name="Food_Menu")

In [4]:
vector_store

## 1. Receipt Generator tool  - Tool number one - Calculate_receipt

In [5]:
def calculate_receipt(user_query):
    order_details = [
    {"food_name": "Pizza", "price": 10.99},
    {"food_name": "Burger", "price": 5.99},
    {"food_name": "Salad", "price": 7.5},
    {"food_name": "Pasta", "price": 8.75},
    {"food_name": "Steak", "price": 15.99},
    {"food_name": "Sushi", "price": 12.5}
    ]
    template = """Given a List containing food_name and price, fetch all the price and add all the prices to return a total price. The list is given in three backticks. 
    ```{order_details}```
    ##User query is: ``{user_query}``
    The output should be of JSON."""

    total_price_calculator_prompt = PromptTemplate(input_variables=["user_query"], template=template)
    total_price_calculator_chain = LLMChain(llm = llm, prompt=total_price_calculator_prompt)
    total_sum = total_price_calculator_chain.run({"user_query":user_query, "order_details": order_details})
    print("receipt_tool")
    return total_sum


receipt_tool = Tool(
        name = "ReceiptCalculator",
        func= calculate_receipt,
        description = "A tool that returns total price when the user asks for receipt or bill."
    )

# # memory = ConversationBufferMemory(memory_key = "chat_history", return_messages= True)

# prompt_receipt = ChatPromptTemplate.from_messages(
#     [
#         (
#             "system",
#             "You are a helpful assistant. Make sure to use the tool for providing receipt for the user.",
#         ),
#         ("placeholder", "{chat_history}"),
#         ("human", "{input}"),
#         ("placeholder", "{agent_scratchpad}"),
#     ]
# )

# # Construct the Tools agent
# # agent = create_tool_calling_agent(llm, receipt_tool, prompt_receipt)

# # agent_executor = AgentExecutor( tools = receipt_tool, agent= agent)
# # agent_executor.invoke({"input":"I need my receipt"})

## 2. Receipe Generator Tool - tool number 2 recipe

In [6]:
# 1. Create a tool
def recipe(user_input):
    
    recipe_retriever = vector_store.as_retriever(search_type = 'mmr', k= 5)
    recipe_template = """### Instruction ###
        You are a good Virtual Assistant . When a user asks you about recipe/ingredients or price of a particular item you have to answer him the recipe or price of the food item based on the context and make sure
        to provide all the details.
        Context is given in triple back ticks: ```{context}```
        Question is given in double back ticks:  ``{question}``
        If the question is out of context, do not provide the wrong answer just say I am sorry i can't answer to that question.
        ### Your Response must be in JSON FORMAT and similar to like this:
        We have multiple options for Sandwich: 
        1.Chicken Breast Sandwich Recipe: "Marinate chicken breasts in olive oil, lemon juice, garlic, and herbs. Grill until cooked through. Toast sandwich buns and assemble with grilled chicken, lettuce, tomato, and mayonnaise. Optionally, add toppings like avocado or bacon. Serve warm for a delicious and wholesome Chicken Breast Sandwich."
    """
    prompt_recipe = PromptTemplate(
        input_variables=["context","question"],
        template=recipe_template
    ) 
    recipe_chain = (
        {"context":recipe_retriever, "question":RunnablePassthrough()}
        | prompt_recipe
        | llm
        | JsonOutputParser()
    )
    print("recipe tool")
    return recipe_chain.invoke(user_input)

recipe_tool = Tool(
    name = "RecipeTool",
    func = recipe,
    description="A Tool that returns recipe of a food"
    
)


# # 2. Prompt for agent
# prompt = ChatPromptTemplate.from_messages(
#     [
#     (
#         "system", 
#         "You are an assistant that returns recipe. Please use the tool for answering the questions. Return the response as specified in tool ."
#     ),
#     ("human","{question}"),
#     ("placeholder","{agent_scratchpad}"),
#     ("placeholder","{tool_names}"),
#     ("placeholder","{tools}"),
#     ]
# )

# # # 3. Initialize an agent, AgentExecutor and run it 
# tools = [recipe_tool]

# agent = create_tool_calling_agent(llm = llm, tools= tools, prompt= prompt)
# agent_executor = AgentExecutor(
#     tools= tools, 
#     agent= agent, 
#     max_iterations= 5, 
#     return_intermediate_steps= False, 
#     verbose=True, 
#     early_stopping_method ='generate')
# ag_response = agent_executor.invoke({"question":"what is the recipe and price of Cali Chicken Sandwich?"})

In [7]:
def chit_chat(user_query):
    first_prompt = ChatPromptTemplate.from_template(
       """You are a waiter at a restaurant. You should have conversation with the user.
        First greet the user and welcome the user to our restaurant and ask him if he would like to order something.
        Explain you need take the food order from the user.
        ### ask_for : ```food_name```
        Current conversation:
        {history}
        Human: {input}
        AI Assistant:
        # """
    )
    
    # info_chain = LLMChain(llm = llm, prompt=first_prompt, verbose=True)
    conversation_chain = ConversationChain(llm= llm, verbose = True,prompt = first_prompt, memory = ConversationBufferMemory(ai_prefix= "AI Assistant"))
    ai_chat = conversation_chain.run(user_query)
    print("Crete order tool")
    return ai_chat

In [8]:
# chit_chat("Hello")

In [9]:
# class OrderFetch(BaseModel):
#     food_name: str = Field(
#         description="This is the food that user will order.",
#     )


# def food_chain_tool(user_query):    
#     food_chain = create_tagging_chain_pydantic(OrderFetch,llm)
#     output =food_chain.run(user_query)
#     return output.food_name


# food_tool = Tool( 
#     func=food_chain_tool,
#     name="foodOrderTool",
#     description="You are an assistant that creates an order. Please use the tool for performing action. Do not look up for answer elsewhere only use the tool to look for answer."
# )



In [10]:
# def record_response(user_query):
#     user_input = input(create_order(user_query))
#     chain = create_tagging_chain_pydantic(OrderFetch,llm)
#     user_response = chain.run(user_input)
#     return user_response
    

In [11]:

# template = """You are a virtual Restaurant Waiter. Your task is to check if the given food_name is present in the context or not. 
# If the food is available in the context and has multiple foods for it, show the user with all the available food names with its price.
# The output should be of JSON FORMAT without any backticks which is given in four backticks return only :
# If not tell them that the option is not available and ask them if they would like to order something else?
# Context is given in three backticks: ```{context}```
# User provided food name is given in double backticks: ``{input}``
# """
# food_verify_tool = create_retriever_tool(
#     vector_store.as_retriever(search_typ'similarity'),
#     name = "OrderCreationTool",
#     description=template
# )
# food_order_prompt = ChatPromptTemplate.from_messages(
#     [
#     (
#         "assistant", 
#         "You are an restaurant waiter that verifies food name. Please use the tool for answering the questions.Do not look up for answer elsewhere only use the tool to look for answer ."
#     ),
#     ("human","{input}"),
#     ("placeholder","{agent_scratchpad}"),
#     ("placeholder","{tool_names}"),
#     ("placeholder","{tools}"),
#     ]
# )

## Tool number 3 - Food verify

In [12]:
# food_name_retriever = vector_store.as_retriever(search_type= 'mmr', kwargs=6)
# food_name_retriever.invoke('Burger')

In [13]:
food_name_retriever = vector_store.as_retriever(search_type= 'mmr', kwargs=6)
# food_name_retriever = vector_store.similarity_search(user_input)
food_template = """You are a virtual Restaurant Waiter. Your task is to check if the given food_name is present in the context or not. 
If the food is available in the context and has multiple foods for it, show the user with all the available food names with its price.
The output should be of JSON FORMAT without any backticks which is given in four backticks return only :
If not tell them that the option is not available and ask them if they would like to order something else?
Context is given in three backticks: ```{context}```
User provided food name is given in double backticks: ``{input}``
"""
prompt_recipe = PromptTemplate(
    input_variables=["context","input"],
    template=food_template
)
chain = RunnableMap({
    "context": lambda x: vector_store.similarity_search(x['input'], k=8),
    "input": lambda x: x['input']
}) | prompt_recipe | llm | JsonOutputParser()

# chain.invoke({"input":"I want to order a burger"})

In [14]:
# chain.invoke({"input":"I want to order a Momo"})

In [15]:
# food_verify_tool = create_retriever_tool(
#     vector_store.as_retriever(search_type = 'similarity'),
#     name = "OrderCreationTool",
#     description=template
# )


def food_verify(user_input):
    print(user_input)
    # food_name_retriever = vector_store.as_retriever(search_type= 'mmr', kwargs=6)
    # food_name_retriever = vector_store.similarity_search(user_input)
    food_template = """You are a virtual Restaurant Waiter. Your task is to check if the given food_name is present in the context or not. 
    If the food is available in the context and has multiple foods for it, show the user with all the available food names with its price.
    The output should be of JSON FORMAT without any backticks which is given in four backticks return only :
    If not tell them that the option is not available and ask them if they would like to order something else?
    Context is given in three backticks: ```{context}```
    User provided food name is given in double backticks: ``{input}``
    """
    prompt_recipe = PromptTemplate(
        input_variables=["context","input"],
        template=food_template
    )
    chain = RunnableMap({
        "context": lambda x: vector_store.similarity_search(x['input'], k=8),
        "input": lambda x: x['input']
    }) | prompt_recipe | llm | JsonOutputParser()

    return chain.invoke({"input":"I want to order a burger"})
# food_chain

food_tool = Tool(
    name = "FoodverifyTool",
    func = food_verify,
    description="A tool that checks for food availability in the context"
    
)



# food_prompt = ChatPromptTemplate.from_messages(
#     [
#     (
#         "assistant", 
#         "You are an restaurant waiter that returns food from the context. Please use the tool for answering the questions.Do not look up for answer elsewhere only use the tool to look for answer ."
#     ),
#     ("human","{input}"),
#     ("placeholder","{agent_scratchpad}"),
#     ("placeholder","{tool_names}"),
#     ("placeholder","{tools}"),
#     ]
# )


# agent = create_tool_calling_agent(llm = llm, tools= [food_tool], prompt= food_prompt)
# agent_executor = AgentExecutor(
#     tools= [food_tool], 
#     agent= agent, 
#     max_iterations= 5, 
#     return_intermediate_steps= False, 
#     verbose=True, 
#     early_stopping_method ='generate')
# ag=agent_executor.invoke({"input":"I want to have a Burger"})
# ag

In [16]:
# food_verify('Burger')

In [17]:
# template = """You are a waiter at a restaurant. Below is are some things to ask the user for in a coversation way. 
#         you should only ask one question at a time even if you don't get all the info 
#         don't ask as a list! First greet the user and welcome the user to our restaurant in a friendly way!
#         Explain you need take the food order from the user.
#         ask_for list: 
#         Current conversation:
#         {history}
#         Human: {input}
#         AI Assistant:
#         """
        
# prompt = PromptTemplate(input_variables=["ask_for","history", "input"], template=template)
# conversation = ConversationChain(llm= llm, verbose = True,prompt = prompt, memory = ConversationBufferMemory(ai_prefix= "AI Assistant"))

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.chains import LLMChain, create_tagging_chain_pydantic
from langchain_core.tools import Tool
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from pydantic import BaseModel, Field
from langchain.memory import ChatMessageHistory
memory = ChatMessageHistory(session_id="test-session")
from langchain.chains import ConversationChain

from llm.model import llm
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser, PydanticOutputParser


class address(BaseModel):
    address:str=Field(description="This is the address of the customers in case of delivery.")
                                                                                              
parser = PydanticOutputParser(pydantic_object=address)

# address_chain=create_tagging_chain_pydantic(address,llm)

def ask_user():
    ask_for=['service']
    first_prompt = ChatPromptTemplate.from_template(
        f'''You are a smart waiter at a restaurant. First  you should ask the user in conversational ways whether the user's order is delivery or dine-in.
        Don not ask addtional information just ask whether the order is delivery or dine-in.
        # Example:"Would you like to have a dine-in or delivery?"
        # ask_for:{ask_for}
        '''
    )   
    chain = LLMChain(llm=llm, prompt=first_prompt, verbose=False)
    conv = chain.run(ask_for=ask_for) 
    return conv

def ask_services():
    service_type=input(ask_user())
    # partial_variables={"format_instructions": parser.get_format_instructions()
    
    
    first_prompt1 = PromptTemplate(
        template= f""" You are a smart assistant waiter in a ABC restaurant.After the order, the user is asked whether he/she wants  delivery or dine-in.Your role is mentioned below:
       
        If the user wants delivery then tell them to provide their delivery address(only in case of delivery) 
       
        If the user wants dine-in then do nothing simply say thank you and enjoy the meal in our restaurant. 
        
        
       Provide staright forward answer related to whether the order is to be deliver or dine-in and do not ask any unwanted additional information
       
       # ask_for :{service_type}
      """     ,
        input_variables=["query"],
        partial_variables={"format_instructions": parser.get_format_instructions()},
    )
    
    chain = LLMChain(llm = llm, prompt=first_prompt1, verbose=False)
    convs = chain.run(ask_for=service_type)
    return convs


def record_address(user_input):
    user_input=input(ask_services())
    chain=create_tagging_chain_pydantic(address,llm)
    response=chain.run(user_input)
    print(response)
    return response

address_tool=Tool(
    name='deliveryaddress',
    func=record_address,
    description=
    "You are a smart assistant waiter in a ABC restaurant. When the user says he has finished his ordering, ask the  whether he/she wants  delivery or dine-in.Your role is mentioned below:\
        If the user wants delivery then tell them to provide their delivery address(only in case of delivery)\
        If the user wants dine-in then do nothing simply say thank you .\
       Provide staright forward answer related to whether the order is to be deliver or dine-in and do not ask any unwanted additional information"  
)


# address_prompt = ChatPromptTemplate.from_messages(
#      [
#     (
#         "system", 
#         "You are a smart assistant waiter in a ABC restaurant.After the order, the user is asked whether he/she wants  delivery or dine-in.Your role is mentioned below:\
#         If the user wants delivery then tell them to provide their delivery address(only in case of delivery)\
#         If the user wants dine-in then do nothing simply say thank you .\
#         Provide staright forward answer related to whether the order is to be deliver or dine-in and do not ask any unwanted additional information"
       
#     ),
#     ("human","{input}"),
#     ("placeholder","{agent_scratchpad}"),
#     ("placeholder","{tool_names}"),
#     ("placeholder","{tools}"),
#     ]
# )

# memory = ChatMessageHistory(session_id="test-session")
# tools = [address_tool]
# agent = create_tool_calling_agent(llm, tools, address_prompt)

# agent_executor = AgentExecutor( tools = tools, agent= agent,return_immediate_steps = False)
# agent_with_chat_history = RunnableWithMessageHistory(
#     agent_executor,
#     # This is needed because in most real world scenarios, a session id is needed
#     # It isn't really used here because we are using a simple in memory ChatMessageHistory
#     lambda session_id: memory,
#     input_messages_key="input",
#     history_messages_key="chat_history",
# )
# # agent_executor.invoke({"input":"I want to have a delivery in Kapan"})

# agent_output = agent_with_chat_history.invoke({"input":"Burger."},
#     config={"configurable": {"session_id": "<foo>"}},
#                                )

In [20]:
chit_chat_tool = Tool(
        name = "ChitChatTool",
        func= chit_chat,
        description = "A tool that does conversation with the user, greets the user and has conversation with the user."
    )

tools = [
    receipt_tool,
    recipe_tool,
    food_tool,
    address_tool,
]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a waiter at a restaurant. You should have conversation with the user.\
            First greet the user and welcome the user to our restaurant and ask him if he would like to order something.\
            Look up for tools to answer the user query do not make up your own answers. Only use tools to answer the user query \
            Always return the response in JSON format as output with double quotation(\") but not single quotation(\') with no \n in the outputs",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

# # Construct the Tools agent
agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    tools= tools, 
    agent= agent, 
    max_iterations= 5, 
    return_intermediate_steps= False, 
    verbose=True, 
    early_stopping_method ='generate')

from langchain.memory import ChatMessageHistory
memory = ChatMessageHistory(session_id="test-session")
from langchain_core.runnables.history import RunnableWithMessageHistory

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)
# output = agent_executor.invoke({"input":"I want to have a Burger"})

# agent_with_chat_history.invoke({"input":"I want to order a Burger"},
#     config={"configurable": {"session_id": "<foo>"}},
#                                )

In [21]:

output = agent_with_chat_history.invoke({"input":"I want to order a Burger"},
    config={"configurable": {"session_id": "<foo>"}},
                               )
print(type(output))
output



> Entering new AgentExecutor chain...

Invoking: `FoodverifyTool` with `Burger`
responded: {"message": "Welcome to our restaurant! Would you like to order something?"}

Burger
{'response': 'Which type of burger would you like to order? Here are the available options:', 'available_options': [{'food_name': 'Cheese Burger', 'price': 13.95}, {'food_name': 'Bacon and Cheddar Burger', 'price': 14.95}, {'food_name': 'Blue cheese Burger', 'price': 17.2}, {'food_name': 'Mushroom cheddar Burger', 'price': 14.95}, {'food_name': 'Mini Burger Sampler', 'price': 14.95}, {'food_name': 'Mexi Burger', 'price': 13.95}, {'food_name': 'Whiskey BBQ Burger', 'price': 15}]}{"message": "Which type of burger would you like to order? Here are the available

> Finished chain.
<class 'dict'>


{'input': 'I want to order a Burger',
 'chat_history': [],
 'output': '{"message": "Which type of burger would you like to order? Here are the available'}

In [23]:
# 'output': '{"message": "Which type of burger would you like to order? Here are the available options:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Mini Burger Sampler", "price": 14.95}, {"food_name": "Mexi Burger", "price": 13.95}, {"food_name": "Whiskey BBQ Burger", "price": 15}]}\n{"message": "Welcome to our restaurant! Would you like to order something?"}'}

In [24]:
agent_with_chat_history.invoke({"input":" I want to order burger and  Momo"},
    config={"configurable": {"session_id": "<foo>"}})



> Entering new AgentExecutor chain...

Invoking: `FoodverifyTool` with `Burger`
responded: {"message": "Which type of burger would you like to order? Here are the available options: Veg Burger, Chicken Burger, or Beef Burger. And for Momo, we have Veg Momo and Chicken Momo. Please specify your choices."}

Burger
{'response': 'Which type of burger would you like to order? We have the following options available: Cheese Burger, Bacon and Cheddar Burger, Blue Cheese Burger, Mushroom Cheddar Burger, Mini Burger Sampler, Mexi Burger, and Whiskey BBQ Burger. Please let me know your choice.'}
Invoking: `FoodverifyTool` with `Momo`


Momo
{'response': 'Which type of burger would you like to order? Here are the available options:', 'available_options': [{'food_name': 'Cheese Burger', 'price': 13.95}, {'food_name': 'Bacon and Cheddar Burger', 'price': 14.95}, {'food_name': 'Blue cheese Burger', 'price': 17.2}, {'food_name': 'Mushroom cheddar Burger', 'price': 14.95}, {'food_name': 'Mini Burger

{'input': ' I want to order burger and  Momo',
 'chat_history': [HumanMessage(content='I want to order a Burger'),
  AIMessage(content='{"message": "Which type of burger would you like to order? Here are the available')],
 'output': '{"message": "Please provide your delivery address for the order."}'}

In [25]:
agent_with_chat_history.invoke({"input":"Do you have fried chicken sandwich?"},
    config={"configurable": {"session_id": "<foo>"}},
                               )




> Entering new AgentExecutor chain...

Invoking: `FoodverifyTool` with `fried chicken sandwich`


fried chicken sandwich
{'response': 'Which type of burger would you like to order? Here are the available options:', 'available_options': [{'food_name': 'Cheese Burger', 'price': 13.95}, {'food_name': 'Bacon and Cheddar Burger', 'price': 14.95}, {'food_name': 'Blue cheese Burger', 'price': 17.2}, {'food_name': 'Mushroom cheddar Burger', 'price': 14.95}, {'food_name': 'Mini Burger Sampler', 'price': 14.95}, {'food_name': 'Mexi Burger', 'price': 13.95}, {'food_name': 'Whiskey BBQ Burger', 'price': 15}]}{"message": "We don't have fried chicken sandwich. Would you like to order something else? Here are the available options for burgers:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Min

{'input': 'Do you have fried chicken sandwich?',
 'chat_history': [HumanMessage(content='I want to order a Burger'),
  AIMessage(content='{"message": "Which type of burger would you like to order? Here are the available'),
  HumanMessage(content=' I want to order burger and  Momo'),
  AIMessage(content='{"message": "Please provide your delivery address for the order."}')],
 'output': '{"message": "We don\'t have fried chicken sandwich. Would you like to order something else? Here are the available options for burgers:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Mini Burger Sampler", "price": 14.95}, {"food_name": "Mexi Burger", "price": 13.95}, {"food_name": "Whiskey BBQ Burger", "price": 15}]}'}

In [26]:
agent_with_chat_history.invoke({"input":"I want to order momo"},
    config={"configurable": {"session_id": "<foo>"}})




> Entering new AgentExecutor chain...
{"message": "Please provide your delivery address for the order."}

> Finished chain.


{'input': 'I want to order momo',
 'chat_history': [HumanMessage(content='I want to order a Burger'),
  AIMessage(content='{"message": "Which type of burger would you like to order? Here are the available'),
  HumanMessage(content=' I want to order burger and  Momo'),
  AIMessage(content='{"message": "Please provide your delivery address for the order."}'),
  HumanMessage(content='Do you have fried chicken sandwich?'),
  AIMessage(content='{"message": "We don\'t have fried chicken sandwich. Would you like to order something else? Here are the available options for burgers:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Mini Burger Sampler", "price": 14.95}, {"food_name": "Mexi Burger", "price": 13.95}, {"food_name": "Whiskey BBQ Burger", "price": 15}]}')],
 'output': '{"message": 

In [40]:
outp  = agent_with_chat_history.invoke({"input":"Suggest me some Shakes"},
    config={"configurable": {"session_id": "<foo>"}})
outp



> Entering new AgentExecutor chain...
{"message": "We have a variety of shakes available. Here are some options:", "available_options": [{"shake_name": "Chocolate Shake", "price": 5.95}, {"shake_name": "Vanilla Shake", "price": 5.95}, {"shake_name": "Strawberry Shake", "price": 5.95}, {"shake_name": "Banana Shake", "price": 5.95}, {"shake_name": "Mango Shake", "price": 5.95}]}

> Finished chain.


{'input': 'Suggest me some Shakes',
 'chat_history': [HumanMessage(content='I want to order a Burger'),
  AIMessage(content='{"message": "Which type of burger would you like to order? Here are the available'),
  HumanMessage(content=' I want to order burger and  Momo'),
  AIMessage(content='{"message": "Please provide your delivery address for the order."}'),
  HumanMessage(content='Do you have fried chicken sandwich?'),
  AIMessage(content='{"message": "We don\'t have fried chicken sandwich. Would you like to order something else? Here are the available options for burgers:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Mini Burger Sampler", "price": 14.95}, {"food_name": "Mexi Burger", "price": 13.95}, {"food_name": "Whiskey BBQ Burger", "price": 15}]}'),
  HumanMessage(content=

In [39]:
output = agent_with_chat_history.invoke({"input":"Cheese Pizza."},
    config={"configurable": {"session_id": "<foo>"}})
import json
json.loads(output.get('output'))




> Entering new AgentExecutor chain...
{"message": "We don't have Cheese Pizza. Would you like to order something else? Here are the available options for burgers:", "available_options": [{"food_name": "Cheese Burger", "price": 13.95}, {"food_name": "Bacon and Cheddar Burger", "price": 14.95}, {"food_name": "Blue cheese Burger", "price": 17.2}, {"food_name": "Mushroom cheddar Burger", "price": 14.95}, {"food_name": "Mini Burger Sampler", "price": 14.95}, {"food_name": "Mexi Burger", "price": 13.95}, {"food_name": "Whiskey BBQ Burger", "price": 15}]}

> Finished chain.


{'message': "We don't have Cheese Pizza. Would you like to order something else? Here are the available options for burgers:",
 'available_options': [{'food_name': 'Cheese Burger', 'price': 13.95},
  {'food_name': 'Bacon and Cheddar Burger', 'price': 14.95},
  {'food_name': 'Blue cheese Burger', 'price': 17.2},
  {'food_name': 'Mushroom cheddar Burger', 'price': 14.95},
  {'food_name': 'Mini Burger Sampler', 'price': 14.95},
  {'food_name': 'Mexi Burger', 'price': 13.95},
  {'food_name': 'Whiskey BBQ Burger', 'price': 15}]}

In [ ]:
{
    "response": "Burger is available in the menu. Here are the available options:",
    "available_options": [
        {
            "food_name": "Cheese Burger",
            "price": 13.95
        },
        {
            "food_name": "Bacon and Cheddar Burger",
            "price": 14.95
        },
        {
            "food_name": "Blue cheese Burger",
            "price": 17.2
        },
        {
            "food_name": "Mushroom cheddar Burger",
            "price": 14.95
        },
        {
            "food_name": "Mini Burger Sampler",
            "price": 14.95
        },
        {
            "food_name": "Mexi Burger",
            "price": 13.95
        },
        {
            "food_name": "Whiskey BBQ Burger",
            "price": 15
        }
    ]
}

def ask_services():
    first_prompt1 = ChatPromptTemplate.from_template(
       """ You are a smart assistant waiter in a ABC restaurant.When the user says he is finished with ordering, Ask the user whether he/she wants delivery or dine-in.Your role is mentioned below:
       Ask the user for their address if the user says delivery. The user input is given in three backticks: ```{input}```
       If the user says dine-in tell them Thankyou and ask no further question.
      """
    )
    chain = LLMChain(llm = llm, prompt=first_prompt1, verbose=True)
    convs = chain.run("delivery")
    return convs

class address(BaseModel):
    address:str=Field(description="This is the address of the customers in case of delivery.")


def record_address(user_input):
    user_input=input(ask_services())
    chain=create_tagging_chain_pydantic(address,llm)
    response=chain.run(user_input)
    print(response)
    return response


delivery_tool=Tool(
    name='deliveryaddress',
    func=record_address,
    description="A tool that asks for address if user says delivery."
)


agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    tools= tools, 
    agent= agent, 
    max_iterations= 5, 
    return_intermediate_steps= False, 
    verbose=True, 
    early_stopping_method ='generate')
agent_executor.invoke({"input":"delivery"})



> Entering new AgentExecutor chain...

Invoking: `deliveryaddress` with `delivery`




> Entering new LLMChain chain...
Prompt after formatting:
Human:  You are a smart assistant waiter in a ABC restaurant.When the user says he is finished with ordering, Ask the user whether he/she wants delivery or dine-in.Your role is mentioned below:
       Ask the user for their address if the user says delivery. The user input is given in three backticks: ```delivery```
       If the user says dine-in tell them Thankyou and ask no further question.
      

> Finished chain.
address='yes'
address='yes'Please provide your delivery address.

> Finished chain.


{'input': 'delivery', 'output': 'Please provide your delivery address.'}